<a href="https://colab.research.google.com/github/aniket-singh-01/SIH2k24/blob/main/yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages
!pip install opencv-python-headless numpy
!apt-get install -y wget unzip
!pip install pdf2image
!apt-get install -y poppler-utils



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
wget is already the newest version (1.21.2-2ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 0s (584 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123597 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up po

In [ ]:
import os

# Define paths
image_dir = '/content/Aadhaar/'  # Directory where your images are stored
annotation_dir = '/content/data/labels/'  # Directory where you want to save annotations

# Ensure the annotation directory exists
os.makedirs(annotation_dir, exist_ok=True)

# Create dummy annotation files for the images
for image_file in os.listdir(image_dir):
    if image_file.endswith('.jpg'):
        base_name = os.path.splitext(image_file)[0]  # Get base name without extension
        annotation_file = os.path.join(annotation_dir, base_name + '.txt')  # Path for annotation file

        # Write dummy annotation
        with open(annotation_file, 'w') as f:
            # Replace this with actual annotation data
            # Format: <class_id> <x_center> <y_center> <width> <height>
            # Placeholder values should be replaced with actual bounding box data
            f.write('0 0.5 0.5 1.0 1.0\n')  # Example dummy annotation (needs real data)


In [ ]:
# Define paths
image_dir = '/content/Aadhaar'
train_file = '/content/data/train.txt'
valid_file = '/content/data/valid.txt'

# Create directories if they don't exist
os.makedirs('/content/data', exist_ok=True)

# Create train.txt and valid.txt with image paths
with open(train_file, 'w') as f_train, open(valid_file, 'w') as f_valid:
    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        f_train.write(f'{image_path}\n')
        f_valid.write(f'{image_path}\n')  # Assuming the same images are used for validation


In [ ]:
# Check contents of train.txt
with open('/content/data/train.txt', 'r') as f:
    print(f.read())

# Check contents of valid.txt
with open('/content/data/valid.txt', 'r') as f:
    print(f.read())


In [ ]:
data_file_content = """
classes= 1
train  = data/train.txt
valid  = data/valid.txt
names  = data/obj.names
backup = backup/
"""
with open('/content/data/aadhar.data', 'w') as f:
    f.write(data_file_content)


In [ ]:
names_file_content = "aadhar_photo"
with open('/content/data/obj.names', 'w') as f:
    f.write(names_file_content)


In [ ]:
import re

def modify_yolo_cfg(cfg_path, output_path, num_classes):
    with open(cfg_path, 'r') as f:
        cfg = f.read()

    # Update filters and classes in the cfg file
    filters = (num_classes + 5) * 3
    cfg = re.sub(r'filters=\d+', f'filters={filters}', cfg)
    cfg = re.sub(r'classes=\d+', f'classes={num_classes}', cfg)

    # Write the modified cfg to a new file
    with open(output_path, 'w') as f:
        f.write(cfg)

# Paths
base_cfg_path = '/content/yolov3.cfg'
modified_cfg_path = '/content/yolov3_custom.cfg'
num_classes = 1

modify_yolo_cfg(base_cfg_path, modified_cfg_path, num_classes)


In [ ]:
with open('/content/yolov3_custom.cfg', 'r') as f:
    content = f.read()
    print(content)


In [ ]:
!cp /content/data/aadhar.data /content/darknet/data/
!cp /content/data/obj.names /content/darknet/data/
!cp /content/yolov3_custom.cfg /content/darknet/
!cp /content/yolov3.weights /content/darknet/


In [ ]:
!git clone https://github.com/pjreddie/darknet
%cd darknet
!make


In [ ]:
# Correct the paths in aadhar.data
data_file_content = """
classes= 1
train  = /content/darknet/data/train.txt
valid  = /content/darknet/data/valid.txt
names  = /content/darknet/data/obj.names
backup = /content/darknet/backup/
"""

# Write the corrected paths to the aadhar.data file
with open('/content/darknet/data/aadhar.data', 'w') as f:
    f.write(data_file_content)


In [ ]:
# Ensure the files are correctly placed
!cp /content/data/train.txt /content/darknet/data/
!cp /content/data/valid.txt /content/darknet/data/
!cp /content/data/obj.names /content/darknet/data/


In [ ]:
# Check contents of train.txt
with open('/content/data/train.txt', 'r') as f:
    print(f.read())

# Check contents of valid.txt
with open('/content/data/valid.txt', 'r') as f:
    print(f.read())


In [ ]:
!cp /content/data/aadhar.data /content/darknet/data/
!cp /content/data/obj.names /content/darknet/data/
!cp /content/yolov3_custom.cfg /content/darknet/
!cp /content/yolov3.weights /content/darknet/


In [ ]:
%cd /content/darknet
!./darknet detector train data/aadhar.data yolov3_custom.cfg yolov3.weights


In [ ]:
# Check the content of aadhar.data
with open('/content/darknet/data/aadhar.data', 'r') as f:
    print(f.read())


In [ ]:
import cv2
import numpy as np
from pdf2image import convert_from_path
import matplotlib.pyplot as plt

def pdf_to_images(pdf_path, dpi=300):
    """
    Convert PDF pages to images.
    Args:
        pdf_path (str): Path to the PDF file.
        dpi (int): Dots per inch for image quality.
    Returns:
        images (list): List of images converted from PDF.
    """
    # Convert PDF pages to images with specified DPI
    images = convert_from_path(pdf_path, dpi=dpi)
    return images

def preprocess_image(image):
    """
    Preprocess the image for rectangle detection.
    Args:
        image (numpy array): Input image.
    Returns:
        thresh (numpy array): Thresholded binary image.
    """
    # Convert to HSV color space for better color processing
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Use the V channel from HSV color space for thresholding
    v_channel = hsv[:, :, 2]

    # Apply Gaussian blur to smooth the V channel image
    blurred = cv2.GaussianBlur(v_channel, (5, 5), 0)

    # Apply adaptive thresholding to get a binary image
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Display the thresholded image for debugging
    plt.imshow(thresh, cmap='gray')
    plt.title('Thresholded Image')
    plt.show()

    return thresh

def find_rectangles(thresh):
    """
    Find rectangular contours in the thresholded image.
    Args:
        thresh (numpy array): Thresholded binary image.
    Returns:
        rectangles (list): List of detected rectangles as (x, y, w, h).
    """
    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    rectangles = []

    for contour in contours:
        # Approximate the contour to a polygon
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        if len(approx) == 4:  # Check if the contour is rectangular
            (x, y, w, h) = cv2.boundingRect(approx)
            aspect_ratio = w / float(h)

            # Filter based on aspect ratio to find rectangular areas
            if 1.2 < aspect_ratio < 2.0:  # Adjust aspect ratio based on your needs
                rectangles.append((x, y, w, h))

    # Print detected rectangles for debugging
    print(f"Detected rectangles: {rectangles}")

    return rectangles

def save_single_extracted_image(image, rectangles, img_index):
    """
    Save a single extracted image from detected rectangles.
    Args:
        image (numpy array): Input image.
        rectangles (list): List of rectangles to extract.
        img_index (int): Index of the image for naming purposes.
    """
    if not rectangles:
        print("No rectangles found to extract.")
        return

    # Select the largest rectangle by area
    largest_rectangle = max(rectangles, key=lambda rect: rect[2] * rect[3])
    x, y, w, h = largest_rectangle

    # Crop the detected object
    extracted_image = image[y:y+h, x:x+w]

    # Save the cropped image
    output_image_path = f'/content/extracted_image_{img_index}.jpg'
    cv2.imwrite(output_image_path, extracted_image)
    print(f"Saved extracted image to {output_image_path}")

    # Display the extracted image for debugging
    plt.imshow(cv2.cvtColor(extracted_image, cv2.COLOR_BGR2RGB))
    plt.title(f'Extracted Image {img_index}')
    plt.show()

# Convert PDF to images
pdf_path = "/content/Masked_Aadhaar.pdf"
images = pdf_to_images(pdf_path, dpi=300)  # Increase DPI for better image quality

# Process the first image (index 0) for demonstration
img_index = 0
img_np = np.array(images[img_index])

# Check image dimensions and color channels for debugging
print(f"Processing image {img_index}: shape {img_np.shape}")

preprocessed_img = preprocess_image(img_np)
rectangles = find_rectangles(preprocessed_img)

# Save the most appropriate extracted image
save_single_extracted_image(img_np, rectangles, img_index)
